## Auto-updating AQI website
#### This website scrapes the data from [India's Pollution Control Board](https://cpcb.nic.in/AQI_Bulletin.php) to map the daily AQIs of major cities

In [48]:
## Imports
from bs4 import BeautifulSoup
import requests
import urllib3
from img2table.document import PDF
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from datetime import datetime
import time

### Scraping the website that has my pdf's url and making it grab the first pdf (the one with the latest AQI) every day

In [49]:
url = "https://cpcb.nic.in/AQI_Bulletin.php"
raw_html = requests.get(url, verify=False).content
soup_doc = BeautifulSoup(raw_html, 'html.parser')
soup_doc.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta content="text/html;charset=utf-8" http-equiv="Content-Type" name="Referrer-Policy" value="no-referrer | same-origin"/>\n  <!--meta charset="utf-8"-->\n  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n  <!--meta charset="utf-8"-->\n  <meta content="cpcb|central pollution control board,aqi,AQI,AQI Delhi,cpcb.nic.in,https://cpcb.nic.in,CPCB,ccb,pollution board,air quality index,cpcb delhi,cpcbmoefcc,pollution control,cpcb,CPCB Water, CPCB Air, CPCB Noise, CPCB Waste, Air Quality Data, Air Pollution control, Air Quality Trends, Water Quality Data, Water Pollution control, Water Quality Trends, Noise Pollution control, Industrial Pollution, CEPI, Critically polluted Industries, Noise Monitoring Data, Environmental Standard, Real time data water air noise,  Annual Reports, Publication, Deepawali, Ambient, SPCB, Kids Corner, Scanned Report, on demand publication, bibliography, glossary, major activities, TSDFs, TSD

In [50]:
# Grabbing first pdf
pdfs = soup_doc.find('ol').find('li').find('a')['href']
pdfs

'https://cpcb.nic.in/displaypdf.php?id=RG93bmxvYWRzL0FRSV9CdWxsZXRpbl8yMDI2MDEyOS5wZGY='

### Naming the file according to the date on which it was uploaded so I know which pdf is being scraped

In [51]:
url = pdfs

current_date = datetime.now()

date_string = current_date.strftime("%Y-%m-%d")

file_name = f"document_{date_string}.pdf"

response = requests.get(url, stream=True, verify=False)

with open(file_name, "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

### After scraping the url that has the pdf and downloading it, I'll import it back again to scrape the pdf using Natural PDF

In [52]:
pdf = PDF(file_name)

In [54]:
#Extracting tables, seeing how many are there
extracted_tables = pdf.extract_tables()

print(f"Found {len(extracted_tables)} tables")

Found 14 tables


In [55]:
extracted_tables[0][0].df #the actual data is nested in a structure like this

,0,1,2,3,4,5
0,S.No,City,Air Quality,Index\nValue,Prominent Pollutant,No. of Stations\nParticipated/\nTotal Stations
1,1,Agra,Moderate,107,PM10,5/6
2,2,Ahmedabad,Moderate,124,"PM10, NO2",8/9
3,3,Ahmednagar,Satisfactory,82,PM10,1/1
4,4,Aizawl,Satisfactory,56,PM2.5,1/1
5,5,Ajmer,Moderate,114,PM10,1/1
6,6,Akola,Satisfactory,72,PM10,1/1
7,7,Alwar,Moderate,123,PM10,1/1
8,8,Amaravati,Poor,227,PM2.5,1/1
9,9,Ambala,Good,50,PM10,1/1


### Looping through all the pages, making each page a df, then joining them all together into `df`

In [56]:
import pandas as pd

dfs = []

for page in extracted_tables.values():
    for table in page:
        # Add it to the list
        dfs.append(table.df)

df = pd.concat(dfs, ignore_index=True)
df.head()

,0,1,2,3,4,5
0,S.No,City,Air Quality,Index\nValue,Prominent Pollutant,No. of Stations\nParticipated/\nTotal Stations
1,1,Agra,Moderate,107,PM10,5/6
2,2,Ahmedabad,Moderate,124,"PM10, NO2",8/9
3,3,Ahmednagar,Satisfactory,82,PM10,1/1
4,4,Aizawl,Satisfactory,56,PM2.5,1/1


### Data cleaning

In [57]:
#Making the first row the header
df.columns = df.loc[0] 
df = df.drop(index=0)

#Each page of the pdf has the headers again and again, I only want them on the first page. Removing them from the rest of the dataframe
df_filtered = df[~df['S.No'].str.contains('S.No', case=False, na=False)] 

#Removing last few unnecessary lines from the df
df = df_filtered[:-7] 

### Making a list of cities to use in the geocoding API and get their lats and longs for mapping. This will be exported as csv and used in a separate notebook, `geocoding.ipynb`, to get each city's coordinates

In [58]:
cities = df[['City']]
cities.to_csv("cities.csv", index=False)

### Reading in the csv I made in the `geocoding.ipynb` here again


In [63]:
new_df = pd.read_csv("city_coords.csv")
new_df = new_df.rename(columns={"city" : "City"})
new_df

,City,Lat,Lng
0,Agra,27.176670,78.008074
1,Ahmedabad,23.022505,72.571362
2,Ahmednagar,19.094829,74.747979
3,Aizawl,23.730718,92.717311
4,Ajmer,26.449895,74.639916
...,...,...,...
237,Virar,19.456360,72.792461
238,Visakhapatnam,17.697380,83.299025
239,Vrindavan,27.565009,77.659339
240,Yadgir,16.748739,77.130872


In [64]:
### Merging the files with the original AQI dfs, so it has Cities and Coordinates

In [65]:
merged_df = pd.merge(df, new_df, on="City", how="left")

#Renaming the df for easier use
df = merged_df

#Verifying count of cities
len(df['City']) 

242

### Saving this as a new csv. This one will be used to power the datawrapper chart.

In [66]:
df.to_csv('final_aqi.csv', index=False)